# Introducción
En este curso profundizaremos lo visto en el anterior, aprendiendo herramientas para mejorar nuestros modelos de Machine Learning:
- Resolver *missing values* y *categorical variables*
- *Pipelines*, mejoran la calidad del código
- *Cross-validation*
- Uso de los últimos Modelos disponibles, *XGBoost*
- Evitar errores comunes como *leakage*

En esta primera sección, no hay mucho para decir, pero sí dejo planteado un ejercicio que había, más que nada porque presenta mejoras al RandomForest, que anteriormente nunca se habían mostrado (*n_estimators*, *depth*)

In [36]:
print('Iniciando...')
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

train_data_filepath = 'assets/input/train.csv'
test_data_filepath = 'assets/input/test.csv'
train_data = pd.read_csv(train_data_filepath)
test_data = pd.read_csv(test_data_filepath)

# target y features de la data de entrenamiento
y = train_data.SalePrice
features = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
X = train_data[features].copy()

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=0, train_size=0.8, test_size=0.2)

# X de la data de test (para submission final)
test_X = test_data[features].copy()

print('Listo.')

Iniciando...
Listo.


In [37]:
# Diferentes variantes de RandomForest (evaluar cual es mejor)
model_1 = RandomForestRegressor(n_estimators=50, random_state=0)
model_2 = RandomForestRegressor(n_estimators=100, random_state=0)
model_3 = RandomForestRegressor(n_estimators=100, criterion='absolute_error', random_state=0)
model_4 = RandomForestRegressor(n_estimators=200, min_samples_split=20, random_state=0)
model_5 = RandomForestRegressor(n_estimators=100, max_depth=7, random_state=0)

models = [model_1, model_2, model_3, model_4, model_5]

In [11]:
# devuelve MAE del modelo indicado
def score_model(model):
    model.fit(train_X, train_y)
    preds = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds)
    return mae

for i in range(len(models)):
    imae = score_model(models[i])
    print('Model_{} - MAE: {:.0f}'.format(i, imae))

Model_0 - MAE: 24015
Model_1 - MAE: 23741
Model_2 - MAE: 23529
Model_3 - MAE: 23997
Model_4 - MAE: 23707


En este caso, el modelo de menor MAE fue el *model_3*, finalmente, entreno dicho modelo con la data completa (train + validation) y genero las predicciones de la *test_data*, para subirlas a la competencia.

In [38]:
# Mejor modelo entre las opciones
model = model_3

model.fit(X, y)
test_preds = model.predict(test_X)

output = pd.DataFrame({'Id': test_X.index, 'SalePrice': test_preds})
output.to_csv('assets/output/home_submission_rforest.csv', index=False)


In [40]:
# no se por que, pero no puedo subir esta submission
# me tira que faltan los elementos (del 1459 a 1469)
# chequeo aca y es verdad, pero la test_data ya me vino sin esos valores (?)
# test_X